In [ ]:
model_name = 'model16'
REM_ANOMS = True
NO_TOP_LOSSES = False
CV = True

import pandas as pd
import numpy as np
import os
import scipy.stats as scs
from IPython.display import display
import gc
import fastai
from fastai.tabular.all import *

import warnings
warnings.filterwarnings('ignore')

pd.options.mode.chained_assignment=None

def pearsonr(x,y): return scs.pearsonr(x,y)[0]

@delegates(AccumMetric)
def PearsonCorrCoef2(dim_argmax=None, **kwargs):
    "Pearson correlation coefficient for regression problem"
    return AccumMetric(pearsonr, invert_arg=False, dim_argmax=dim_argmax, **kwargs)

In [ ]:
DATASET = 'ubi-14-15-16-17'
KFOLDS = 20
PREFIX = 'learner_'
SUFFIX= '_BEST_EPOCH'
learners = [load_learner(f"../input/{DATASET}/{PREFIX}{model_name}_fold_{i}{SUFFIX}.pkl", cpu=False) for i in range(1,KFOLDS + 1)]
# for learner in learners: 
#     learner.path = Path('.')
#     learner.model_dir = Path('.')

# learners = [learner.load(f'../input/ubi-model11-cv/{model_name}_fold_{i+1}', with_opt=False, strict=False) for i,learner in enumerate(learners)]

all_preds = pd.DataFrame()

with open("../input/ds-ubi1/anom_feats.pkl", "rb") as f:
    anom_feats = pickle.load(f)
    
cont = [f"f_{i}" for i in range(300)]
labels = cont
labels.append('investment_id')
# labels = [x for x in labels if x not in anom_feats]

# labels

In [ ]:
import ubiquant
env = ubiquant.make_env()  # initialize the environment
iter_test = env.iter_test()  # an iterator which loops over the test set and sample submission


for (test_df, sample_prediction_df) in iter_test:
    _test_df = test_df[labels]  #.drop(anom_feats, axis=1)  # drop the anomalous features    
    _preds = pd.DataFrame()
    
    for i,learn in enumerate(learners):
        with learn.no_bar():
            dl = learn.dls.test_dl(_test_df)
            y, _ = learn.get_preds(dl=dl)
        
        _preds[str(i)] = y.numpy().flatten()
        
    preds = _preds.mean(axis=1)  # average the predictions
    preds = preds.tolist()
    display(type(preds), _preds, preds)
    
    sample_prediction_df["target"] = preds  # make your predictions here
    sample_prediction_df.fillna(0)
#     display(sample_prediction_df)
    env.predict(sample_prediction_df)  # register your predictions

In [ ]:
blah = pd.read_csv('submission.csv')
display(blah.shape, blah)

In [ ]:
blah['target'].dtype

In [ ]:
test_df

In [ ]:
sample_prediction_df

In [ ]:
type(y.numpy())

In [ ]:
blah.iloc[8,1]